# 문서 분류 모델

**문서 분류**
- 문서가 주어졌을 때 해당 문서의 범주를 분류하는 과제

**사용할 데이터**
- 네이버 영화 리뷰 말뭉치(NSMC) 
- 영화 리뷰 문장을 입력으로 하고 해당 문장이 속한 극성의 확률 출력
- 확률 값을 적당한 후처리 과정을 거쳐 긍정, 부정처럼 사람이 보기에 좋은 형태로 가공
- 감성 분석 : 문장의 극성을 분류하는 과제

**모델 구조**
- 입력 문장을 토큰화한 뒤 문장 시작과 끝을 알리는 스페셜 토큰 CLS와 SEP를 각각 원래 토큰 시퀀스 앞뒤에 붙임
- 이를 BERT 모델에 입력하고 문장 수준의 벡터(pooler_output)을 뽑음
- 이 벡터에 작은 추가 모듈을 덧붙여 모델 전체의 출력이 [해당 문장이 긍정일 확률, 해당 문장이 부정일 확률] 형태가 되도록 함

**태스크 모듈**
- pooler_output 벡터 뒤에 붙은 추가 모듈
- 드롭아웃, 가중치 행렬 곱, 소프트맥스 함수 등
- 파인튜닝

# 영화 리뷰 감성 분석 모델 만들기

In [ ]:
# 의존성 패키지 설치 
!pip install ratsnlp

# 모델 환경 설정 
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base", # 프리트레인 마친 언어 모델 이름 (허깅페이스 모델 허브에 등록 필)
    downstream_corpus_name="nsmc", # 다운스트림 데이터 이름
    # downstream_corpus_root_dir= "" # 다운스트림 데이터를 내려받을 위치. 입력하지 않으면 코랩 환경 로컬에 저장 
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls", # 파인튜닝된 모델의 체크포인트가 저장될 위치 
    batch_size=32 if torch.cuda.is_available() else 4, # 배치 크기
    learning_rate=5e-5, # 러닝 레이트. 1회 스텝에서 모델을 얼마나 업데이트 할지
    max_seq_length=128, # 토큰 기준 입력 문장 최대 길이 
    epochs=3, # 학습 에포크 수
    tpu_cores=0 if torch.cuda.is_available() else 8, # TPU 코어 수
    seed=7, # 랜덤 시드 
)

# 랜덤 시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

# 로그 출력 로거 설정
nlpbook.set_logger(args)

In [ ]:
# 말뭉치 내려 받기 
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=True,
)

In [ ]:
# 토크나이저 준비
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

- 딥러닝 모델을 학습하려면 학습 데이터를 배치 단위로 계속 모델에 공급 해야 함
- 파이토치에서는 이 역할을 데이터 로더가 수행
- 데이터 로더는 데이터셋이 보유하고 있는 인스턴스를 배치 크기만큼 뽑아서 자료형, 데이터 길이 등 정해진 형식에 맞춰 배치를 만들어 줌

In [ ]:
# 데이터 전처리
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset
corpus = NsmcCorpus() # csv 파일 형식의 NSMC 데이터를 문장과 레이블로 읽어 들임 
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

# 학습용 데이터 로더 구축. 학습 때 배치 구성은 랜덤으로 하는 것이 좋음 
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)

# 평가용 데이터 로더 구축 
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

In [ ]:
# 모델 초기화 
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)

model = BertForSequenceClassification.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

In [ ]:
# task 정의
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)

# 트레이너 정의
trainer = nlpbook.get_trainer(args)

# 학습 개시
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

# 인퍼런스

**인퍼런스**
- 학습을 마친 모델로 실제 과제를 수행하는 행위나 그 과정 
- 모델을 문서 분류라는 실전에 투입

**웹 서비스**
- 네트워크에서 컴퓨터 간에 상호작용을 하기 위해 만들어진 소프트웨어 시스템
- 원격 사용자가 보낸 문장을 수신해 해당 문장이 긍정인지 부정인지 응답을 만들고 이 응답을 원격 사용자에게 전달하는 웹 서비스 구축

In [ ]:
# 의존성 패키지 설치
!pip install ratsnlp

# 인퍼런스 설정
from ratsnlp.nlpbook.classification import ClassificationDeployArguments
args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls",
    max_seq_length=128,
)

In [ ]:
# 토크나이저 로드
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

# 체크포인트 로드
import torch
from transformers import BertConfig, BertForSequenceClassification
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)

# BERT 설정 로드 
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)
model = BertForSequenceClassification(pretrained_model_config) # BERT 모델 초기화 
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()}) # 체크포인트 주입
model.eval() # 평가 모드 전환 

In [ ]:
# 인퍼런스 함수 정의
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()}) # 파이토치 텐서로 변경 
        prob = outputs.logits.softmax(dim=1) # 로짓에 소프트맥스 
        positive_prob = round(prob[0][1].item(), 4) # 긍정/부정 확률 소수점 4자리로 반올림
        negative_prob = round(prob[0][0].item(), 4) # 긍정/부정 확률 소수점 4자리로 반올림
        pred = "긍정 (positive)" if torch.argmax(prob) == 1 else "부정 (negative)" # 예측 확률의 최댓값 위치에 따라 pred 만들기 
    return {
        'sentence': sentence,
        'prediction': pred,
        'positive_data': f"긍정 {positive_prob}",
        'negative_data': f"부정 {negative_prob}",
        'positive_width': f"{positive_prob * 100}%",
        'negative_width': f"{negative_prob * 100}%",
    }

In [ ]:
# 웹 서비스 시작
# ngrok은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구입니다. ngrok을 실행하려면 회원가입 후 로그인을 한 뒤 이곳에 접속해 인증 토큰(authtoken)을 확인해야 합니다. 예를 들어 확인된 authtoken이 test111이라면 다음과 같이 실행합니다.
!mkdir /root/.ngrok2 && echo "authtoken: {이곳에 확인된 인증 토큰을 입력하세요}" > /root/.ngrok2/ngrok.yml

from ratsnlp.nlpbook.classification import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()